In [3]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

2024-10-06 09:12:12.292212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 09:12:12.305405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 09:12:12.308955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 09:12:12.320155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-06 09:12:13.217665: W tensorflow/compiler/tf2

In [2]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1728198737.663221   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198738.138820   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198738.139078   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
BATCH_SIZE=32

#### Train/test sets

In [4]:
# Train set : CIFAKE/train
# Class FAKE : CIFAKE/train/FAKE
# Class REAL : CIFAKE/train/REAL
train_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/train', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

# Test set : CIFAKE > test
# Class FAKE : CIFAKE/test/FAKE
# Class REAL : CIFAKE/test/REAL
validation_set = tf.keras.utils.image_dataset_from_directory(
    'CIFAKE/test', 
    seed=42,
    image_size=(32,32),
    batch_size=BATCH_SIZE
)

Found 100000 files belonging to 2 classes.


I0000 00:00:1728198758.629844   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.630060   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.630214   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728198758.729480   16465 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 20000 files belonging to 2 classes.


In [5]:
class_names = train_set.class_names
print(f'Training classes: {class_names}')

class_names = validation_set.class_names
print(f'Validation classes: {class_names}')

Training classes: ['FAKE', 'REAL']
Validation classes: ['FAKE', 'REAL']


#### Image normalisation

In [6]:
# Image normalisation

#### Build model

In [7]:
# Build model
def CNN_model01():
    model = Sequential([
        Rescaling(1./255),
        Conv2D(32, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),

        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer='adam', loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()]                  
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [8]:
CNN_model01().summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       460,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 461,825 (1.76 MB)

 Trainable params: 461,825 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

#### Train model

In [13]:
N_EPOCHS = 10

In [14]:
time_start = dt.now()
print(f'Start training, time: {time_start.time()}')

model = CNN_model01()

history = model.fit(
  train_set,
  validation_data=validation_set,
  epochs=N_EPOCHS,
  verbose=1
)

print(f'Time elapsed: {dt.now() - time_start}')

Start training, time: 09:14:30.596175
Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.8123 - loss: 0.3982 - precision_2: 0.8041 - recall_2: 0.8233 - val_accuracy: 0.8943 - val_loss: 0.2572 - val_precision_2: 0.8992 - val_recall_2: 0.8883
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8965 - loss: 0.2493 - precision_2: 0.8926 - recall_2: 0.9007 - val_accuracy: 0.9153 - val_loss: 0.2120 - val_precision_2: 0.9313 - val_recall_2: 0.8966
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9206 - loss: 0.1966 - precision_2: 0.9188 - recall_2: 0.9221 - val_accuracy: 0.9110 - val_loss: 0.2189 - val_precision_2: 0.8727 - val_recall_2: 0.9624
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9327 - loss: 0.1704 - precision_2: 0.9305 - recall_2: 0.9350 - val_accuracy: 0.9182 - val_loss: 0.2065 - val_precision_2: 0.9013 - val_recall_2: 0.9392
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9432 - loss: 0

#### Visualisation

In [15]:
pd.DataFrame(history.history)

,accuracy,loss,precision_2,recall_2,val_accuracy,val_loss,val_precision_2,val_recall_2
0,0.85447,0.331957,0.846616,0.86580,0.89435,0.257182,0.899180,0.8883
1,0.90456,0.233907,0.901397,0.90850,0.91525,0.212042,0.931339,0.8966
2,0.92381,0.190191,0.922433,0.92544,0.91100,0.218926,0.872688,0.9624
3,0.93488,0.164620,0.932785,0.93730,0.91815,0.206502,0.901257,0.9392
4,0.94518,0.140714,0.942930,0.94772,0.92365,0.199576,0.921752,0.9259
5,0.95246,0.122059,0.950927,0.95416,0.92700,0.199681,0.911924,0.9453
6,0.95954,0.105494,0.958111,0.96110,0.92635,0.212194,0.919512,0.9345
7,0.96507,0.089798,0.963448,0.96682,0.92910,0.211797,0.920439,0.9394
8,0.97026,0.077347,0.968966,0.97164,0.92425,0.242837,0.924972,0.9234
9,0.97429,0.068261,0.973069,0.97558,0.92320,0.261844,0.918347,0.9290


In [27]:
hist = history.history


train_col = 'precision_2'
val_col = f'val_{train_col}'

fig = make_subplots(rows=2, cols=2, subplot_titles=("Loss over epochs", f"{train_col} over epochs"))

# Loss
fig.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

# 
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=1, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=1, col=2)

#
train_col = 'recall_2'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=1 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=1)

train_col = 'accuracy'
val_col = f'val_{train_col}'
fig.add_trace(  go.Scatter(x=list(range(len(hist[train_col]))), y=hist[train_col],  mode='lines+markers', name=f'Training {train_col}'),  row=2, col=2 )
fig.add_trace( go.Scatter(x=list(range(len(hist[val_col]))), y=hist[val_col], mode='lines+markers', name=f'Validation {train_col}'), row=2, col=2)


fig.update_xaxes(title_text="Epochs", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=1, col=2)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=1)
fig.update_yaxes(title_text=f"{train_col}", row=2, col=2)

fig.update_layout(
    # title_text="Training and Validation Metrics over Epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

In [12]:
# Confusion matrix
# conf_matrix = confusion_matrix(y_test, y_pred_classes)

#### Class activation map